In [51]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,398 kB]
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packages [1,136

In [52]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-17 07:05:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.44MB/s    in 0.2s    

2023-03-17 07:05:38 (4.44 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [53]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [54]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
appliances_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep= ('\t'), header=True, inferSchema=True)

# Show DataFrame
appliances_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [55]:
#check columns in dataframe
column_names = appliances_data_df.columns
column_names

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [56]:
# Get the number of rows in the DataFrame.
appliances_data_df.count()


85981

# Transform the Data

## Create the "review_id_table".

In [57]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = appliances_data_df.select(['review_id', 'customer_id','product_id','product_parent',to_date('review_date','yyyy-mm-dd').alias('review_date')])
review_id_df.show() # will only show 20 rows


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1QX6706ZWJ1P5|   32114233|B00OYRW4UE|     223980852| 2015-01-31|
|R3QWMLJHIW6P37|   18125776|B0000537JQ|     819771537| 2015-01-31|
|R14Z1VR1N0Z9G6|   19917519|B00HXXO332|     849307176| 2015-01-31|
| R25ZRJL0GH0U0|   18277171|B00EOB0JA2|     700864740| 2015-01-31|
|R3837KYH7AZNIY|    2593270|B00OC2O1UC|     794298839| 2015-01-31|
|R2MN0QYCY6EVIV|    2592955|B00HES9CMS|     318730927| 2015-01-31|
|R3AN2UJ1D42ZS0|   15168265|B0016BFR4G|     887476137| 2015-01-31|
|R3U29ZLUWEEK4M|   13761624|B00K504UUG|     458706868| 2015-01-31|
|R16ZDMJJHK796C|   37070734|B00HES9CMS|     318730927| 2015-01-31|
| RRRDOEJZD1Y22|   29615023|B00P6TUO5G|     170248843| 2015-01-31|
|R2KR5ZEAS859DK|   47893062|B0006VJ6TO|     412568457| 2015-01-31|
| RR7PGQY763IHF|    2582596|B00H9L7VIW|     851045898| 2015-01

In [58]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## Create the "products" Table

In [59]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = appliances_data_df.select(['product_id', 'product_title'])
products_df.show() # will only show 20 rows

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00OYRW4UE|Elite Sportz Exer...|
|B0000537JQ|     Ezy Dose Weekly|
|B00HXXO332|Pulse Oximeter, B...|
|B00EOB0JA2|SE Tools Tool Kit...|
|B00OC2O1UC|doTERRA HD Clear ...|
|B00HES9CMS|Viva Naturals #1 ...|
|B0016BFR4G|Uncle Lee's Organ...|
|B00K504UUG|Syrtenty Electrod...|
|B00HES9CMS|Viva Naturals #1 ...|
|B00P6TUO5G|Viva Naturals Org...|
|B0006VJ6TO|Body Back Company...|
|B00H9L7VIW|boostULTIMATE - 6...|
|B00P1JNZUW|doTERRA Veráge Sk...|
|B00N5HD340|      Straight Razor|
|B0007DHMVK|BONGER Massage To...|
|B0002JG2NI|Home Health Casto...|
|B00OYRW4UE|Elite Sportz Exer...|
|B00598WT0C|Aloha Eyewear “St...|
|B0006VJ6TO|Body Back Company...|
|B00J7G96NK|Oral B Glide Flos...|
+----------+--------------------+
only showing top 20 rows



In [60]:
#Drop duplicates
products_df = products_df.drop_duplicates()

## Create the "customers" Table

In [61]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = appliances_data_df.groupby('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   11160821|    1|
|    1099354|    1|
|   37200466|    1|
|   31581262|    1|
|   27596904|    1|
|    2405598|    1|
|   11229098|    1|
|   17237226|    1|
|   46000147|    1|
|   15234640|    1|
|   28937218|    1|
|   43920023|    1|
|   27954895|    1|
|    8269764|    1|
|   31689770|    1|
|   30117841|    1|
|   38591899|    1|
|   24557051|    1|
|    2780908|    1|
|   32466409|    1|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [62]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df= appliances_data_df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|
|R3QWMLJHIW6P37|          5|            0|          0|   N|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|
| R25ZRJL0GH0U0|          2|            0|          0|   N|
|R3837KYH7AZNIY|          4|            0|          1|   N|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|
|R16ZDMJJHK796C|          5|            0|          0|   N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



# Load

In [64]:
mode = "append"
jdbc_url="jdbc:postgresql://postgres.ccxnnnes366j.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "******", 
          "driver":"org.postgresql.Driver"}

In [65]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [66]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [67]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [68]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)